In [10]:
import requests
import json
import pandas as pd
import numpy as np

from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


In [3]:
#liste des 35 meilleures villes  selectionnée selon One Week In.com
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens",
               "Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon",
               "Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille",
                "Aix en Provence","Avignon","Uzes",
                "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne",
                "Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [ ]:
# Créer une liste pour stocker les ID des villes
id_list = [i + 1 for i in range(len(cities))]

# Créer un DataFrame pour les données météorologiques
weather = pd.DataFrame(columns=['id', 'city', 'lat', 'lon', 'datetime', 'temp_min', 'temp_max', 'humidity', 'wind_speed', 'weather_main', 'weather_description'])

for idx, city in enumerate(cities):
    url = 'https://nominatim.openstreetmap.org/search?q='
    suffix = '&format=geojson'
    r = requests.get(url + city + suffix)
    lat = str(r.json()['features'][0]['geometry']['coordinates'][1])
    lon = str(r.json()['features'][0]['geometry']['coordinates'][0])

    urlweather = "http://api.openweathermap.org/data/2.5/forecast?lat="
    s1 = '&lon='
    units = "units=metric&" 
    s2 = '&cnt='
    cnt = str(20)  
    s3 = '&'
    API = "APPID=e80e40a61ae4e01bc0889eb3d63c2d5a"
    q = requests.get(urlweather + lat + s1 + lon + s2 + cnt + s3 + units + API)
    weather_data = q.json()

    for i in range(20):  # 
        timestamp = weather_data['list'][i]['dt']
        temp_min = weather_data['list'][i]['main']['temp_min']
        temp_max = weather_data['list'][i]['main']['temp_max']
        weather_desc = weather_data['list'][i]['weather'][0]['description']
        weather_main = weather_data['list'][i]['weather'][0]['main']
        humidity = weather_data['list'][i]['main']['humidity']
        wind_speed = weather_data['list'][i]['wind']['speed']

        
    
        temp = q.json()['list'][i]['main']['temp']  

        new_record = {'id': id_list[idx], 'city': city, 'lat': float(lat), 'lon': float(lon), 'datetime': datetime.fromtimestamp(timestamp),
                      'temp_min': temp_min, 'temp_max': temp_max, 'humidity': humidity, 'wind_speed': wind_speed,
                      'weather_main': weather_main, 'weather_description': weather_desc}

        weather.loc[len(weather)] = new_record


In [ ]:
r

<Response [200]>

In [ ]:
display(weather)

,id,city,lat,lon,datetime,temp_min,temp_max,humidity,wind_speed,weather_main,weather_description
0,1,Mont Saint Michel,48.635954,-1.511460,2023-12-02 21:00:00,1.30,1.59,85,5.47,Clouds,overcast clouds
1,1,Mont Saint Michel,48.635954,-1.511460,2023-12-03 00:00:00,1.53,2.00,81,6.36,Rain,light rain
2,1,Mont Saint Michel,48.635954,-1.511460,2023-12-03 03:00:00,2.86,3.64,85,8.08,Clouds,overcast clouds
3,1,Mont Saint Michel,48.635954,-1.511460,2023-12-03 06:00:00,3.73,3.73,96,7.56,Clouds,overcast clouds
4,1,Mont Saint Michel,48.635954,-1.511460,2023-12-03 09:00:00,5.08,5.08,94,6.69,Rain,light rain
...,...,...,...,...,...,...,...,...,...,...,...
695,35,La Rochelle,46.159113,-1.152043,2023-12-04 18:00:00,10.62,10.62,66,11.49,Rain,light rain
696,35,La Rochelle,46.159113,-1.152043,2023-12-04 21:00:00,9.98,9.98,62,10.26,Rain,light rain
697,35,La Rochelle,46.159113,-1.152043,2023-12-05 00:00:00,9.77,9.77,63,8.46,Clouds,broken clouds
698,35,La Rochelle,46.159113,-1.152043,2023-12-05 03:00:00,9.98,9.98,70,9.57,Rain,light rain


In [ ]:
weather = weather.drop_duplicates(subset=['lat', 'lon'])
weather = weather.reset_index(drop=True)

In [ ]:
weather.head()

,id,city,lat,lon,datetime,temp_min,temp_max,humidity,wind_speed,weather_main,weather_description
0,1,Mont Saint Michel,48.635954,-1.511460,2023-12-02 21:00:00,1.30,1.59,85,5.47,Clouds,overcast clouds
1,2,St Malo,49.314695,-96.953823,2023-12-02 21:00:00,0.55,1.58,45,6.89,Clouds,overcast clouds
2,3,Bayeux,49.276462,-0.702474,2023-12-02 21:00:00,0.14,0.48,69,4.48,Clouds,overcast clouds
3,4,Le Havre,49.493898,0.107973,2023-12-02 21:00:00,4.07,4.51,70,6.16,Clear,clear sky
4,5,Rouen,49.440459,1.093966,2023-12-02 21:00:00,0.57,0.93,87,2.79,Clouds,scattered clouds


In [ ]:
weather['temp_average'] = weather[['temp_max', 'temp_min']].mean(axis=1).round(0)
weather.head()

,id,city,lat,lon,datetime,temp_min,temp_max,humidity,wind_speed,weather_main,weather_description,temp_average
0,1,Mont Saint Michel,48.635954,-1.511460,2023-12-02 21:00:00,1.30,1.59,85,5.47,Clouds,overcast clouds,1.0
1,2,St Malo,49.314695,-96.953823,2023-12-02 21:00:00,0.55,1.58,45,6.89,Clouds,overcast clouds,1.0
2,3,Bayeux,49.276462,-0.702474,2023-12-02 21:00:00,0.14,0.48,69,4.48,Clouds,overcast clouds,0.0
3,4,Le Havre,49.493898,0.107973,2023-12-02 21:00:00,4.07,4.51,70,6.16,Clear,clear sky,4.0
4,5,Rouen,49.440459,1.093966,2023-12-02 21:00:00,0.57,0.93,87,2.79,Clouds,scattered clouds,1.0


In [ ]:
weather = weather.sort_values(by='temp_average', ascending=False)

In [ ]:
weather['weather_main'].unique()

array(['Clouds', 'Clear', 'Snow'], dtype=object)

In [ ]:
weather
weather.to_csv('weather_data.csv', index=False)

In [ ]:
top_five_weather = weather.head(5)
top_five_weather

,id,city,lat,lon,datetime,temp_min,temp_max,humidity,wind_speed,weather_main,weather_description,temp_average
27,28,Collioure,42.525050,3.083155,2023-12-02 21:00:00,6.80,8.46,78,9.56,Clouds,few clouds,8.0
33,34,Bayonne,43.494514,-1.473666,2023-12-02 21:00:00,6.49,7.12,93,2.67,Clouds,broken clouds,7.0
32,33,Biarritz,43.483252,-1.559278,2023-12-02 21:00:00,7.37,7.62,93,2.11,Clouds,broken clouds,7.0
25,26,Aigues Mortes,43.566152,4.191540,2023-12-02 21:00:00,4.24,4.84,45,5.55,Clear,clear sky,5.0
18,19,Bormes les Mimosas,43.150697,6.341928,2023-12-02 21:00:00,4.23,4.60,50,11.92,Clear,clear sky,4.0


In [8]:
top_five_weather=pd.read_csv('top_five_weather.csv')

In [6]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter_mapbox(top_five_weather, lat="lat", lon="lon", color="temp_average", size="temp_average", hover_name="city", zoom=6, mapbox_style="carto-positron")
fig.show()

In [ ]:
top_five_weather = top_five_weather['city']
top_five_weather.to_string('top_five_weather.txt',index = False)

In [ ]:
top_five_weather


27             Collioure
33               Bayonne
32              Biarritz
25         Aigues Mortes
18    Bormes les Mimosas
Name: city, dtype: object